In [ ]:
#merge(df1,df2,left_on...)

In [1]:
#I NEED TO KNOW THE WAY IT WAS ANONYMISED BECAUSE IF THEY WERE DONE THE SAME THEN i CAN MATCH ID OR NAME
#IF NOT THEN MAYBE DO IT PER REGION AND THEN DISTRICT AND THEN WHITTLE IT DOWN?

In [2]:
#I matched columns by eye, seeing which values were similar and labelled region, district, province etc
#maybe make a set and see which values are in the same set?
#df['id'].equals(df1['id'])

In [57]:
#import packages
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [58]:
dum_file_path = r'C:\\Users\\Penelope\\Documents\\ASSIGNMENT\\dummy.xlsx'

In [59]:
dum = pd.read_excel (dum_file_path)

In [60]:
dum.head()

heading missing number form.health_centre_information.facility_name  \
0        0            ---                             William hospital   
1        1              1                         Kathryn Hospital/CHC   
2        2            NaN                             Jessica Hospital   
3        3              3                                   Lauren CHC   
4        4            NaN                                    Tammy CHC   

  form.health_centre_information.location_information.district  \
0                                            Pujehun             
1                                                 Bo             
2                                                 Bo             
3                                             Bonthe             
4                                             Bonthe             

  form.health_centre_information.facility_type  \
0                        community_health_post   
1                        community_health_post   
2                        community_health_post   
3                                     mch_post   
4                        community_health_post   

  form.health_centre_information.setting  \
0                             peri_urban   
1                                  rural   
2                                  rural   
3                             peri_urban   
4                             peri_urban   

  form.health_centre_information.outpatient_only  \
0                                            yes   
1                                            yes   
2                                            yes   
3                                             no   
4                                             no   

   missing form.health_centre_information.capacity.number_consultation_rooms  \
0                                                NaN                           
1                                                1.0                           
2                                                NaN                           
3                                                1.0                           
4                                                NaN                           

   form.health_centre_information.capacity.number_inpatient_beds  \
0                                                  0               
1                                                  1               
2                                                  1               
3                                                  2               
4                                                  1               

   form.health_centre_information.capacity.number_maternity_beds  ...  \
0                                                  1              ...   
1                                                  2              ...   
2                                                  1              ...   
3                                                  3              ...   
4                                                  1              ...   

   form.grp_infection_prevention_and_control.ql_water_sanitation_and_hygiene.score_max_infection_prevention_and_control_water_sanitation_and_hygiene  \
0                                                  5                                                                                                   
1                                                  5                                                                                                   
2                                                  5                                                                                                   
3                                                  5                                                                                                   
4                                                  5                                                                                                   

   form.grp_infection_prevention_and_con

In [61]:
#missing values
total_cells = np.product(dum.shape)
total_cells

864

In [62]:
missing_values = dum.isnull().sum()
print(missing_values.sum())


5


In [63]:
#awesome
col_list = dum.columns.to_list()
col_list[-1]

'% score'

In [64]:
from sklearn.model_selection import train_test_split

In [65]:
#select target
y = dum["% score"]

In [66]:
#just checking it worked
#print(y)

In [67]:
dum_predictors = dum.drop(['% score'], axis=1)

In [68]:
#exclude non numerical data 
X = dum_predictors.select_dtypes(exclude=['object'])

In [47]:
#print(X)

In [69]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [70]:
#print(X_train)

In [71]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [51]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
5.392156862745098


In [72]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
6.384803921568625


In [73]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))


KeyError: 'missing number'